# Homework:

---

**NAME: Yubo Cai**

**User ID: yubuo.cai@polytechinique**

---

**Instructions:** 
- The clarity (clean writing) and the organization of the work (numbering the questions appropriately) are taken into account during the correction. 
- Before uploading your notebook, **Kernel/restart and clear output**. And **verify that your code is running cell after cell.**
- This homework is **optional**. It counts at most as 4 **bonus** points on the final grade (over 20) of the course (capped at 20). It must be uploaded back on moodle **BEFORE** the 12th of March at 8PM (Paris time) to be counted, no delay will be accepted. 

---

In [ ]:
# Load packages:

# this package allows to work efficiently with arrays
import numpy as np
# this package is used to draw graphs
import matplotlib.pyplot as plt
# for the numerical application in the end, we will use pi
from math import pi

## Exercise

Consider the matrices 

$$A = \left(\begin{array}{ccc} 1 & 0 & 0 \\ 2 & 3 & 0 \\ 4 & 5 & 6\end{array}\right), \qquad 
  B = \left(\begin{array}{ccc} 1 & 2 & 1 \\ 2 & 5 & 4 \\ 1 & 4 & 6\end{array}\right), $$
and the vector $b = \left(\begin{array}{c} 1 \\ 1 \\ 1\end{array}\right)$.

**Question 1.1.**

Write down the steps of the forward substitution algorithm to solve the problem $AV = b$. 


**Answer**: 

Since the matrix $A$ is already in the lower triangular form and therefore invertible. Also, the values in diagonal are nonzero. So we can directly apply the forward substitution algorithm with the following algorithms:

- Interations: For $i$ from 1 to $N$ (here N is 3)

  $$
  V_{i} = \frac{b_{i}-\sum_{j-1}^{i-1}A_{i,j}V_{j}}{A_{i,i}}
  $$

Therefore, we have the following program to solve the problem $AV = b$, we have the final result:
$$
V= \begin{pmatrix} 1 \\ -\frac{1}{3} \\ -\frac{2}{9} \end{pmatrix}
$$

In [ ]:
def forward_substitution(L, b):
    V = np.ones(len(b))
    N = len(b)
    for i in range(1, N + 1):
        V[i - 1] = (b[i - 1] -
                    np.dot(L[i - 1, 0:i - 1], V[0:i - 1])) / L[i - 1, i - 1]

    return V


# result
L = np.array([[1, 0, 0], [2, 3, 0], [4, 5, 6]])
b1 = np.array([1, 1, 1])

sol1 = forward_substitution(L, b1)
print("solution V = ", sol1)
print("L V = ", np.matmul(L, sol1), "  b = ", b1)


**Question 1.2.**

Write down the steps of the Cholesky algorithm to decompose the matrix $B = L L^T$. 

**Answer**: 

We apply the Cholesky algorithm with the following algorithms:

- Iterations: For $i$ from 1 to $N$ (here N is 3)

  - We first compute values in diagonal
  $$
  L_{i,i} = \sqrt{B_{i,i}-\sum_{k=1}^{i-1}L_{i,k}^2}
  $$

  - Then, for i from i to N, we have
  $$
  L_{i,j} = \frac{B_{i,j}-\sum_{k=1}^{j-1}L_{i,k}L_{j,k}}{L_{j,j}}, \quad \text{for } i>j
  $$

However, for the above algorithms, in order to compute the diagonal, we need to know the values in the column, so more precisely, we need to compute $L$ **column by column from left to right**, we start from the first column.

- We first compute the first diagonal point in the first column with formula:
  $$
  L_{1,1} = \sqrt{B_{1,1}}
  $$
  Then we can compute the rest points in the first column
  $$
  L_{i,1} = \frac{B_{i,1}}{L_{1,1}}, \quad \text{for } i>1
  $$

- For the second column, we also **first compute the diagonal point** in the columns with the formula we have above, then we compute the rest points in the columns.

- We apply the similar algorithms for the third column and rest columns.

Therefore, we have the following program to solve the problem $B = L L^T$, we have the final result:
$$
L = \begin{pmatrix} 1 & 0 & 0 \\ 2 & 1 & 0 \\ 1 & 2 & 1 \end{pmatrix}
$$



In [ ]:
def Cholesky_decomp(B):
    N = len(B[0, :])
    L = np.eye(N)

    for i in range(N):
        for j in range(i):
            L[i, j] = (B[i, j] - np.sum([L[i, k] * L[j, k]
                                         for k in range(j)])) / L[j, j]
        L[i,
          i] = np.sqrt(B[i, i] - np.sum([L[i, k] * L[i, k] for k in range(i)]))

    return L


# result
B = np.array([[1, 2, 1], [2, 5, 4], [1, 4, 6]])
# implement your test here

print("Matrix:\n", B, "\n")
print("Cholesky decomposition:\n", Cholesky_decomp(B))
L = Cholesky_decomp(B)
print("L L^T =\n ", np.matmul(L, np.transpose(L)))

## Problem

**Definition:** A matrix $S$ is orthogonal if it is invertible and its inverse satisfies $S^{-1} = S^T$. 

The goal of this tutorial is to construct an algorithm providing a decomposition of a matrix $A = S U$ into the product of an orthogonal matrix $S$ and an upper triangular one $U$. 

---

### Preliminary computations on orthogonal matrices

**Question 2.1.**

a) Consider an orthogonal matrix $S$. What values can $det(S)$ have?

b) Are all matrices satisfying this property orthogonal? Prove it or give a counterexample. 

**Answer:** 

a) We have the definition of orthogonal matrix $S$ is that $S^{-1} = S^T$. Also we know that $det(S^{-1}) = \frac{1}{det(S)}$. Therefore, we have the equation that
  $$
  det(S^{-1}) = det(S^T) \Rightarrow det(S) = \frac{1}{det(S)} \Rightarrow det(S)^2 = 1
  $$
  Therefore, we have $det(S) = \pm 1$

b) We have the matrix $A = \begin{pmatrix} 1 & 0 \\ 2 & 1 \end{pmatrix}$, where $det(A)=1$. However, we have $A^{-1}= \begin{pmatrix} 1 & 0 \\ -2 & 1 \end{pmatrix} \neq  \begin{pmatrix} 1 & 2 \\ 0 & 1 \end{pmatrix}=A^{T}$. Therefore, we have the counterexample that not all matrices satisfying the property of orthogonal matrix $S$ are orthogonal.


**Question 2.2.**

Prove that the product of orthogonal matrices remains an orthogonal matrix. 

**Answer:** 

We assume that matrix $A \in \mathbb{R}^{N \times N}$ and $B \in \mathbb{R}^{N \times N}$ are orthogonal matrices. Then obviously $A$ and $B$ are invertible. We also have the property that $A^{T}=A^{-1}$ and $B^{T}=B^{-1}$. We have the following equation:
$$
\begin{aligned}
(AB)^{T} &= B^{T} A^T \\
&= B^{-1} A^{-1} \\
&= (AB)^{-1}
\end{aligned}
$$
For the property $AB^{-1} = B^{-1}A^{-1}$, we have following proof: By the definition of inverse matrix:
$$
\mathbf{A} \mathbf{A}^{-1}=\mathbf{A}^{-1} \mathbf{A}=\mathbf{I}
$$
and
$$
\mathbf{B B}^{-1}=\mathbf{B}^{-1} \mathbf{B}=\mathbf{I}
$$
Now, observe that:
$$
\begin{array}{rlr}
(\mathbf{A B})\left(\mathbf{B}^{-1} \mathbf{A}^{-1}\right) & =\left(\mathbf{A}\left(\mathbf{B} \mathbf{B}^{-1}\right)\right) \mathbf{A}^{-1} & \text { (Matrix Multiplication is Associative) } \\
& =(\mathbf{A I}) \mathbf{A}^{-1} & \\
& =\mathbf{A} \mathbf{A}^{-1} & \\
& =\mathbf{I} & \text { (Definition of Identity Element) }
\end{array}
$$
Similarly:
$$
\begin{array}{rlr}
\left(\mathbf{B}^{-1} \mathbf{A}^{-1}\right)(\mathbf{A B}) & =\left(\mathbf{B}^{-1}\left(\mathbf{A}^{-1} \mathbf{A}\right)\right) \mathbf{B} \quad \text { (Matrix Multiplication is Associative) } \\
& =\left(\mathbf{B}^{-1} \mathbf{I}\right) \mathbf{B} & \\
& =\mathbf{B}^{-1} \mathbf{B} \quad \text { (Definition of Identity Element) } \\
& =\mathbf{I}
\end{array}
$$
Then we prove the property that the product of orthogonal matrices remains an orthogonal matrix.


**Question 2.3.**

Prove that the Euclidean norm of a vector is preserved when multiplying it by an orthogonal matrix $S$, i.e. 

$$\|S V\| = \|V\|$$

where the Euclidean norm yields $\|V\| = \sqrt{V^T V}$.

**Answer:** 

We have
$$
\begin{aligned}
\|SV\| &= \|(SV)^{T}SV\| \\
&= \|V^{T}S^{T}SV\| \\
&= \|V^{T}S^{-1}SV\|\quad \text{Since $S$ is orthogonal matrix} \\
&= \|V^{T}V\| \\
&= \|V\|
\end{aligned}
$$
Then we finish the proof.


**Question 2.4.**

Consider a vector $V \in \mathbb{R}^N$, and define the matrix 

$$ S(V) = Id - 2 \frac{V V^T}{\|V\|^2},$$ 

where $\|V\| = \sqrt{V^T V}$ denotes the Euclidean norm of $V$. 

Prove that $S(V)$ is an orthogonal matrix. 

**Answer:**  

- We first prove that $S(V)$ is square, since $V \in \mathbb{R}^{N}$, therefore we have $VV^{T} \in \mathbb{R}^{N, N}$. Then we have $S(V) \in \mathbb{R}^{N, N}$ is square.

- Then we try to prove $S(V)S(V)^{T}=I$. We have for $S(V)^{T}$:
  $$
  S(V)^{T} = Id^{T} - 2 \frac{(V^{T} V)^{T}}{\|V\|^2} = Id - 2 \frac{(V^{T})^{T} V^{T}}{\|V\|^2} = Id - 2 \frac{V V^T}{\|V\|^2} = S(V)
  $$
  Then we can compute $S(V)S(V)^{T}$:
  $$
  \begin{aligned}
  S(V)S(V)^{T} & = S(V)^2 \\
  &= (Id - 2 \frac{V V^T}{\|V\|^2})^2 \\
  &= Id - 4 \frac{V V^T}{\|V\|^2} + 4 \frac{(V V^T)^{2}}{\|V\|^4} \\
  &= Id - 4 \frac{V V^T}{\|V\|^2} + 4 \frac{V V^T V V^T}{\|V\|^4} \\
  &= Id - 4 \frac{V V^T}{\|V\|^2} + 4 \frac{ \|V\|^2 V V^T}{\|V\|^4} \\
  &= Id - 4 \frac{V V^T}{\|V\|^2} + 4 \frac{V V^T}{\|V\|^2} \\
  &= Id
  \end{aligned}
  $$

- Finally, we need to prove that $S(V)$ is invertible, we can prove this by computing its determinant
  $$
  det(S(V)) = det(Id - 2 \frac{V V^T}{\|V\|^2}) =  1 - 2 \frac{V^T V}{\|V\|^2} = 1 - 2 \frac{\|V\|^2}{\|V\|^2} =  -1 \neq 0
  $$
  Then we prove that $S(V)$ is invertible.

Therefore, we prove that $S(V)$ is an orthogonal matrix.

---

### Construction of the algorithm (QR decomposition based on Householder reflections)

As in the Gaussian elimination algorithm, we eliminate the subdiagonal coefficients of a matrix $A$. But, instead of using elementary matrices, we use orthogonal matrices $S(V)$ as defined in 4).  

Let us rewrite $A = (C^1, C^2, \dots, C^N)$ as a row vector of column vectors $C^i \in \mathbb{R}^N$. We multiply iteratively the matrix $A$, and therefore each of its column, by an orthogonal matrix $S(V^i)$.  

**Question 2.5.**

Consider $C^1 \in \mathbb{R}^N$, we seek a vector $V^1 \in \mathbb{R}^N$ such that 

$$S(V^1) C^1 = \left( \begin{array}{c} \alpha \\ 0 \\ \vdots \\ 0 \end{array} \right). \qquad{} (1) $$ 

What are the possible values of $\alpha$? If there are several possibilities, choose one of them here for the rest of the algorithm. 

**Answer:** 

We assume the equation 
$$S(V^1) C^1 = \left( \begin{array}{c} \alpha \\ 0 \\ \vdots \\ 0 \end{array} \right)$$ 
Therefore, we have the following equation:
$$
\begin{aligned}
S(V^1) C^1 &= \begin{pmatrix} \alpha \\ 0 \\ \vdots \\ 0 \end{pmatrix}   \\
(Id - 2 \frac{V^{1}(V^{1})^{T}}{\|V^{1}\|^2}) C^{1} &= \alpha e_{1} \\
2\frac{(C^{1})^{T}V^{1}}{\|V^{1}\|^2} V^{1} &= C^{1} - \alpha e_{1} \\
\end{aligned}
$$
Since we know that $2\frac{(C^{1})^{T}V^{1}}{\|V^{1}\|^2}$ is a constant, we can rewrite the equation as:
$$
V^{1} = m(C^{1} - \alpha e_{1}) \quad \text{where $m$ is a constant}
$$h
In order to apply the elimination, we have $\alpha = \pm \|C^1\|_{2}$ and $V = m(C^1 \pm \|C^1\|_{2}e_{1})$ where $m$ is any constant and **we choose** $\alpha = \|C^1\|_{2}$.

**Question 2.6.**

We seek $V^1\in\mathbb{R}^N$ satisfying (1) and such that $\|V^1\| = 1$. Let us decompose $V^1 = \left( \begin{array}{c} a \\ W \end{array} \right)$ and $C^1 = \left( \begin{array}{c} b \\ X \end{array} \right)$.

a) Prove that $W = kX$ is proportional to $X$. 

b) Write $a$ as a function of $k$ and $X$.

c) Write an equation satisfied by $k$ depending only on $X$ and $b$. 

d) Solve this equation and compute $k$.  

**Answer:** 

a) From previous question, we choose $\alpha=\|C^1\|_{2}$. Therefore, we have the following equation:
$$
\begin{aligned}
(Id-2(V^{1})^{T}V^{1}) C^{1} &= \|C^1\|_{2} e_{1} \\
C^{1} - 2 (V^{1})^{T}V^{1} C^{1} &= \|C^1\|_{2} e_{1}\\
C^{1} - 2 ((C^{1})^{T} V^{1})V^{1} &= \|C^1\|_{2} e_{1}\\
\end{aligned}
$$
Since $(C^{1})^{T} V$ is a constant, we can rewrite the equation as:
$$
V^{1} = m(C^{1} - \|C^1\|_{2} e_{1}) \quad \text{where $m$ is a constant}
$$
Therefore we have the relation that
$$
\begin{pmatrix} a \\ W \end{pmatrix} = m \begin{pmatrix} b-\|C^1\|_{2} \\ X \end{pmatrix} 
$$
Therefore, here $m$ is $k$, we prove that $W = kX$ is proportional to $X$.

b) Since we have $\|V^{1}\|=1$, therefore, we have
$$
\|V^{1}\|^2 = a^{2}+\|W\|^2 =1
$$
Also, from the previous equation, we have
$$
W = kX \Rightarrow a^{2}+k^{2}\|X\|^2 =1 
$$
Therefore, we can deduce that
$$
a^{2} = 1-k^{2}\|X\|^2 \Rightarrow a = \sqrt{1-k^{2}\|X\|^2}
$$

c) From previous equation, we have
$$
\begin{cases}
a = k(b-\|C^1\|_{2})\\
W = kX
\end{cases}
$$
For the first equation, we got
$$
a = k(b-\sqrt{b^{2}+\|X\|^{2}}) \Rightarrow \sqrt{1-k^{2}\|X\|^2} = k(b-\sqrt{b^{2}+\|X\|^{2}})
$$
Therefore, by take the square on both side, we have:
$$
1-k^{2}\|X\|^2 = k^{2}(b-\sqrt{b^{2}+\|X\|^{2}})^{2}
$$

d) From the equation in c), we have:
$$
1 = k^{2} (\|X\|^2 + (b-\sqrt{b^{2}+\|X\|^{2}})^{2}) \Rightarrow k = \frac{1}{\sqrt{\|X\|^2 + (b-\sqrt{b^{2}+\|X\|^{2}})^{2}}}
$$
We can represent $k$ with $X$ and $b$:
$$
k = \frac{1}{\sqrt{\|X\|^2 + (b-\sqrt{b^{2}+\|X\|^{2}})^{2}}}
$$
**Or we can express $k$ with $\alpha$, we have**:
$$
k = \frac{1}{\sqrt{\|X\|^2 + (b-\sqrt{b^{2}+\|X\|^{2}})^{2}}} = \frac{1}{\sqrt{\|X\|^2+b^{2}-2b\sqrt{b^{2}+\|X\|^{2}}+\|X\|^2}+b^2} = \frac{1}{\sqrt{2(b^2+\|X\|^2)-2b\sqrt{b^2+\|X\|^2}}}
$$
Therefore, we subsitute $\alpha$ with $\sqrt{b^2+\|X\|^2}$:
$$
k = \frac{1}{\sqrt{2 \alpha (\alpha - b)}}
$$
Where $\alpha = \sqrt{b^2 + \|X\|^2}$. **Both expressions of $k$ with $k=\frac{1}{\sqrt{\|X\|^2 + (b-\sqrt{b^{2}+\|X\|^{2}})^{2}}}$ and $k = \frac{1}{\sqrt{2 \alpha (\alpha - b)}}$ with $\alpha = \sqrt{b^2 + \|X\|^2}$ should be the same**.

**Question 2.7.**

a) What vector operations do you need to perform to compute $S(V^1) C^2$ for some other vector $C^2 \in\mathbb{R}^N$? 

b) How many scalar operations are therefore necessary to compute this product?

c) How many scalar operations are therefore necessary to compute the product $S(V^1) A$, without performing the trivial operations?

**Answer:** 

a) From the previous formola, we got 
$$
S(V^1) C^2 = (Id - 2 \frac{V^{1}(V^{1})^{T}}{\|V\|^2}) C^{2} 
$$
Since we already know that $\|V^{1}\|=1$. Therefore, we have 
$$
\begin{aligned}
S(V^1) C^2 &= (Id - 2 \frac{V^{1}(V^{1})^{T}}{\|V\|^2}) C^{2} \\
&=  (Id - 2 V^{1}(V^{1})^{T})C^{2} \\
&= C^{2} - 2 V^{1}(V^{1})^{T} C^{2} \\
\end{aligned}
$$

Therefore, we have **4 vector operations** in total, which are:
- 1 vector transpose: $(V^{1})^{T}$
- 2 vector multiplication: $V^{1}(V^{1})^{T}$ and multiplication between $V^{1}(V^{1})^{T}$ and $C^{2}$
- 1 vector subtraction: $C^{2} - 2 V^{1}(V^{1})^{T} C^{2}$

Since $\|V^1\|^2$ is a constant, the division operation is not vector operation. Therefore, we have **4 vector operations** in total.


b) Since we have:
$$
S(V^1) C^2 = C^{2} - 2 V^{1}(V^{1})^{T} C^{2}
$$

Therefore, we have **n scalar operations** in total, which is:
- n scalar multiplication: since we know that $V^{1}(V^{1})^{T} C^{2} \in \mathbb{R}^n$, therefore, we have **n scalar multiplication**.

c) Since $A = (C^1, C^2, \cdots, C^n)$ is a matrix containing $n$ vectors, therefore, we have:
$$
n \times n \text{ scalar operation} = n^2 \text{ scalar operations}
$$
Therefore, for each vector $C^i$, we need to perform **n scalar operation**. Therefore, we have **$n^{2}$ scalar operations** in total.

**Question 2.8.**

Now, we need to eliminate the subdiagonal term on the second column without modifying the first column of $A^1 := S(V^1)A$, i.e. we want 

$$S(V^2) \tilde{C}^2 = \left( \begin{array}{c} * \\ \beta \\ 0 \\ \vdots \\ 0 \end{array} \right).$$

where $\tilde{C}^i$ is the i-th column of $A^1$.

For this purpose, we use another matrix $S(V^2)$ satisfying 

$$ S(V^2)_{1,i} = \delta_{1,i}. $$

What does this condition imposes on the entries of the vector $V^2$ ?

**Answer (Description of the algorithms):** 
After the decomposition with $S(V^1)C^1$, we got:
$$
A^{1}=S(V^1) A=\left[\begin{array}{c|ccc}
\alpha & \tilde{a}_{12} & \cdots & \tilde{a}_{1 n} \\
\hline 0 & & & \\
\vdots & & \tilde{A}_2 & \\
0 & & &
\end{array}\right]
$$
Then we try to eliminate the subdiagonal term on the second column without modifying the first column of $A^1 := S(V^1)A$, we can apply the same method to the $\tilde{A}_2 \in \mathbb{R}^{(n-1) \times (n-1)}$. We try to find a householder matrix $\tilde{H}_2 \in \mathbb{R}^{(n-1) \times (n-1)}$ to elminate all the numbers in the first column of $\tilde{A}_2$ to 0 as follow:
$$
\tilde{H}_2 \tilde{A}_2=\left[\begin{array}{c|ccc}
\beta & \tilde{a}_{23} & \cdots & \tilde{a}_{2 n} \\
\hline 0 & & & \\
\vdots & & \tilde{A}_3 & \\
0 & & &
\end{array}\right] .
$$
where we got
$$
S(V^2) =\left[\begin{array}{cc}
1 & 0 \\
0 & \tilde{H}_2
\end{array}\right] \text {. }
$$
where $S(V^2) \in \mathbb{R}^{n \times n}$ and we have 
$$
S(V^2) S(V^1) A=\left[\begin{array}{c|ccc}
\alpha & \tilde{a}_{12} & \cdots & \tilde{a}_{1 n} \\
\hline 0 & & & \\
\vdots & & \tilde{H}_2 \tilde{A}_2 & \\
0 & & &
\end{array}\right]=\left[\begin{array}{cc|ccc}
\alpha & \tilde{a}_{12} & \tilde{a}_{13} & \cdots & \tilde{a}_{1 n} \\
0 & \beta & \tilde{a}_{23} & \cdots & \tilde{a}_{2 n} \\
\hline 0 & 0 & & & \\
\vdots & \vdots & & \tilde{A}_3 & \\
0 & 0 & & &
\end{array}\right] .
$$
And the algorithm is applied interatively in this way.

---

**Answer to the Question 2.8:**

Since $S(V^2)$ is a orthogonal matrix, we have $S(V^2)_{1,i} = \delta_{1,i}$ and $S(V^2)_{i,1} = \delta_{1,i}$. Therefore, the matrix $S(V^2)$ should be in this format:
$$
S(V^2) =\left[\begin{array}{cc}
1 & 0 \\
0 & \tilde{H}_2
\end{array}\right]
$$
We also have: 
$$
S(V^2) \tilde{C}^2 = \left( \begin{array}{c} * \\ \beta \\ 0 \\ \vdots \\ 0 \end{array} \right).
$$
Therefore, we need to find a vector $V^2 \in \mathbb{R}^N$ such that $S(V^2) =\left[\begin{array}{cc}
1 & 0 \\
0 & \tilde{H}_2
\end{array}\right]$. From the previous formola, we got:
$$
S(V^2) = (Id - 2 \frac{V^{2}(V^{2})^{T}}{\|V\|^2}) = \left[\begin{array}{cc}
1 & 0 \\
0 & \tilde{H}_2
\end{array}\right]
$$. 
Therefore, in order to make $S(V^2)_{1,1}=1=Id_{1,1}$, we got $V^2=\begin{pmatrix}0 \\ v_2 \\ \vdots \\ v_n \end{pmatrix}$. Next, we try to compute the part of $V^2$ for $v_2$ to $v_n$. This question is equivalent to how can we decompose to $\tilde{H}_2$ such that $
\tilde{H}_2 \tilde{A}_2=\left[\begin{array}{c|ccc}
\beta & \tilde{a}_{23} & \cdots & \tilde{a}_{2 n} \\
\hline 0 & & & \\
\vdots & & \tilde{A}_3 & \\
0 & & &
\end{array}\right] 
$. We can see that this problem is the exactly same with the **Question 2.6**. Therefore, we can use the same method to solve this problem.

We denote a vector $V' \in \mathbb{R}^{n-1} = \begin{pmatrix} v_2 \\ \vdots \\ v_n \end{pmatrix}$. From **Question 2.5**, we have the formula for $V'$ such that
$$
V' = \tilde{C}^2_{sub}- \|\tilde{C}^2_{sub}\|_{2} e_{1}
$$
where we denote $\tilde{C}^2_{sub} = \begin{pmatrix} \tilde{a}_{22} \\ \vdots \\ \tilde{a}_{n 2} \end{pmatrix}$ and $e_1 ' \in \mathbb{R}^{n-1} = \begin{pmatrix} 1 \\ 0 \\ \vdots\\ 0 \end{pmatrix}$. Therefore, we can conclude that 
$$
V^2 = \begin{pmatrix} 0 \\ V' \end{pmatrix} = \begin{pmatrix} 0 \\ \tilde{a}_{22} - \|\tilde{C}^2_{sub}\|_{2} \\ \tilde{a}_{32} \\ \vdots \\ \tilde{a}_{n2}  \end{pmatrix}
$$
where $\tilde{C}^2_{sub} = \begin{pmatrix} \tilde{a}_{22} \\ \vdots \\ \tilde{a}_{n 2} \end{pmatrix}$ 

**Question 2.9.**

Again, what is the value of $\beta$ ? And propose a vector $V^2 \in \mathbb{R}^N$ satisfying $\|V^2\| = 1$ and (2).


**Answer:** 

From the conclusion of **Question 2.5** and the algorithms of **Question 2.8**, we choose $\alpha = \|C^1\|$, for the same reason, we choose $\beta = \|\tilde{C}^2_{sub}\|$. Therefore, we have:
$$
\beta = \|\tilde{C}^2_{sub}\| = \sqrt{\tilde{a_{22}}^2 + \cdots  +\tilde{a_{n2}}^2} = \sqrt{\|\tilde{C}^2\|-\tilde{a_{12}}^2}
$$
where **$\tilde{a_{22}}, \cdots, \tilde{a_{n2}}$ are from $\tilde{C}^2 = \text{i-th column of }A^{1} = \begin{pmatrix} \tilde{a}_{12} \\ \tilde{a}_{22} \\ \vdots \\ \tilde{a}_{n 2} \end{pmatrix}$** 

---

In order to satisfy $\|V^2\| = 1$, we set 
$$
A^{1} = \begin{pmatrix} \tilde{a}_{12} \\ \tilde{a}_{22} \\ X' \end{pmatrix} \quad \text{and} \quad V^{2} = \begin{pmatrix} 0 \\ a' \\ W' \end{pmatrix}
$$
We apply the same algorithm as **Question 2.6** to compute $V^{2}$. We have:
$$
b' = \tilde{a}_{22}
$$
From **Question 2.6 d)**, we have:
$$
k' = \frac{1}{2\beta (\beta - \tilde{a}_{22})} 
$$
Then we got
$$
a' = \sqrt{1-(k')^2 \|X'\|^2}
$$
and
$$
W' = k' X'
$$
In conclusion, we got the vector of $V^{2}$:
$$
V^{2} =  \begin{pmatrix} 0 \\ \sqrt{1-(k')^2  \|X'\|^2} \\  k' X' = k' \begin{pmatrix} \tilde{a}_{32} \\ \vdots \\ \tilde{a}_{n2} \end{pmatrix} \end{pmatrix}
$$
where $X' = \begin{pmatrix} \tilde{a}_{32} \\ \vdots \\ \tilde{a}_{n2} \end{pmatrix}$, $k' = \frac{1}{2\beta (\beta - \tilde{a}_{22})}$ and $\beta = \|\tilde{C}^2_{sub}\| = \sqrt{\tilde{a_{22}}^2 + \cdots  +\tilde{a_{n2}}^2}$

In [ ]:
from pprint import pprint

def qr_householder(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()

    for j in range(n):
        # Apply Householder transformation to eliminate below-diagonal entries
        x = R[j:, j]
        norm_x = np.linalg.norm(x)
        v = np.zeros_like(x)
        v[0] = np.copysign(norm_x, x[0])
        v += x
        v /= np.linalg.norm(v)

        R[j:, :] -= 2 * np.outer(v, np.dot(v, R[j:, :]))
        Q[:, j:] -= 2 * np.outer(Q[:, j:], np.dot(Q[:, j:].T, v))

    return Q.T, R

---

In practice, this technique is applied iteratively to all columns until writing a product of the form 

$$\left(\prod\limits_i S(V^i) \right) A = U $$

which provides the decomposition $A = SU$ where $S = \left(\prod\limits_i S(V^i) \right)^T$.